# 16-way version (annotation version)

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time


In [2]:
# Define a function to return HOG features and visualization
from skimage.feature import hog
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features


In [3]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import local_binary_pattern
from sklearn.externals import joblib
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

# Define a function to return LBP features
def get_lpb_features(img):
    return np.ravel(local_binary_pattern(img, 16, 2, method='uniform'))

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, cells=True, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)
        
        if cells: # Already sized at 16x16 cells
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                    #hog_features.append(np.concatenate((get_hog_features(feature_image[:,:,channel], 
                    #                    orient, pix_per_cell, cell_per_block, 
                    #                    vis=False, feature_vec=True), get_lpb_features(feature_image[:,:,channel]))))
                    #hog_features.append(feature_image[:,:,channel])
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            features.append(hog_features)
            
        else:
            #first, slice the 64x64 image into quads by column
            quad= []
            quad.append(feature_image[:, 0:16,:])
            quad.append(feature_image[:,16:32,:])
            quad.append(feature_image[:,32:48,:])
            quad.append(feature_image[:,48:64,:])
            for i in range(4):
                #next, slice the quad into cells(16x16) by row
                square= []
                square.append(quad[i][ 0:16,:,:])
                square.append(quad[i][16:32,:,:])
                square.append(quad[i][32:48,:,:])
                square.append(quad[i][48:64,:,:])
                for j in range(4):
                    feature_image= square[j]
                    # Call get_hog_features() with vis=False, feature_vec=True
                    if hog_channel == 'ALL':
                        hog_features = []
                        for channel in range(feature_image.shape[2]):
                            hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                                orient, pix_per_cell, cell_per_block, 
                                                vis=False, feature_vec=True))
                            #hog_features.append(np.concatenate((get_hog_features(feature_image[:,:,channel], 
                            #                    orient, pix_per_cell, cell_per_block, 
                            #                    vis=False, feature_vec=True), get_lpb_features(feature_image[:,:,channel]))))
                            #hog_features.append(feature_image[:,:,channel])
                        hog_features = np.ravel(hog_features)        
                    else:
                        hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                                    pix_per_cell, cell_per_block, vis=False, feature_vec=True)
                    # Append the new feature vector to the features list
                    features.append(hog_features)
            del quad, square
    # Return list of feature vectors
    return features

# load filenames of images containing vehicles
cars = []
vimages = glob.glob('analysis/copycar/processed/carcells/*.png')
for image in vimages:
    cars.append(image)

# load filenames of images containing non-vehicles
nimages = glob.glob('training/non-vehicles/Extras/*.png')
notcars = []
for image in nimages:
    notcars.append(image)

del vimages, nimages
# Reduce the sample size because HOG features are slow to compute
#sample_size = 500
#cars = cars[0:sample_size]
#notcars = notcars[0:sample_size]
print('Step complete.')

Step complete.


In [4]:
### TODO: Tweak these parameters and see how the results change.
colorspace = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 12
pix_per_cell = 4
cell_per_block = 2
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"

t=time.time()
print('Extracting cells of cars...')
car_features = extract_features(cars, cells=True, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
print('Extracting notcars images...')
notcar_features = extract_features(notcars, cells=False, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)

# load additional cells containing non-vehicles
nimages = glob.glob('analysis/copycar/processed/notcarcells/*.png')
notcars = []
for image in nimages:
    notcars.append(image)
print('Extracting cells of notcars...')
notcarcell_features = extract_features(notcars, cells=True, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
del nimages

t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')

print(np.array(car_features).shape, np.array(notcar_features).shape, np.array(notcarcell_features).shape)
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features, notcarcell_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)
print('Updating scaler.dat')
joblib.dump(X_scaler, 'scalar.dat')  # save for later use by classifier

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features)), np.zeros(len(notcarcell_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.1, random_state=rand_state)

del scaled_X, y, cars, notcars
del car_features, notcar_features, notcarcell_features
print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))


Extracting cells of cars...
Extracting notcars images...
Extracting cells of notcars...
31.85 Seconds to extract HOG features...
(1874, 1296) (15488, 1296) (1761, 1296)
Updating scaler.dat
Using: 12 orientations 4 pixels per cell and 2 cells per block
Feature vector length: 1296


In [8]:
# Use a linear SVC 
#svc = LinearSVC()
from sklearn import svm
svc = svm.SVC(kernel='poly', gamma=2)

# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('   The SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

1169.09 Seconds to train SVC...
Test Accuracy of SVC =  0.9493
   The SVC predicts:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
For these 10 labels:  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0.18752 Seconds to predict 10 labels with SVC


In [20]:
#####################################################################################################################
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

#model= Sequential([
#    Dense(1296, input_dim=len(X_train[0]), init='normal', activation='relu'),
#    Dropout(.5),
#    Dense(1, init='normal', activation='sigmoid')
#])

model= Sequential([
    Dropout(.5, input_shape=(len(X_train[0]),) ),
    Dense(512, init='normal', activation='relu'),
    Dropout(.5),
    Dense(1, init='normal', activation='sigmoid')
])

# binary classification
model.compile(optimizer='adam',  #rmsprop
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dropout_3 (Dropout)              (None, 1296)          0           dropout_input_1[0][0]            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 512)           664064      dropout_3[0][0]                  
____________________________________________________________________________________________________
dropout_4 (Dropout)              (None, 512)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 1)             513         dropout_4[0][0]                  
Total params: 664,577
Trainable params: 664,577
Non-trainable params: 0
___________________

In [21]:
# Check the training time for the model
t=time.time()
model.fit(X_train, y_train, nb_epoch=28, batch_size=32, shuffle=True, verbose=2)
t2 = time.time()

print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(model.evaluate(X_test, y_test, batch_size=16)[1], 4))


Epoch 1/28
4s - loss: 0.3716 - acc: 0.8865
Epoch 2/28
4s - loss: 0.2213 - acc: 0.9182
Epoch 3/28
4s - loss: 0.1907 - acc: 0.9278
Epoch 4/28
4s - loss: 0.1750 - acc: 0.9378
Epoch 5/28
4s - loss: 0.1561 - acc: 0.9440
Epoch 6/28
4s - loss: 0.1488 - acc: 0.9463
Epoch 7/28
4s - loss: 0.1288 - acc: 0.9549
Epoch 8/28
4s - loss: 0.1305 - acc: 0.9552
Epoch 9/28
4s - loss: 0.1167 - acc: 0.9587
Epoch 10/28
4s - loss: 0.1174 - acc: 0.9594
Epoch 11/28
4s - loss: 0.1099 - acc: 0.9618
Epoch 12/28
3s - loss: 0.1060 - acc: 0.9633
Epoch 13/28
3s - loss: 0.1036 - acc: 0.9656
Epoch 14/28
3s - loss: 0.0991 - acc: 0.9676
Epoch 15/28
3s - loss: 0.0942 - acc: 0.9684
Epoch 16/28
3s - loss: 0.0911 - acc: 0.9705
Epoch 17/28
3s - loss: 0.0952 - acc: 0.9695
Epoch 18/28
3s - loss: 0.0914 - acc: 0.9714
Epoch 19/28
3s - loss: 0.0840 - acc: 0.9726
Epoch 20/28
3s - loss: 0.0803 - acc: 0.9730
Epoch 21/28
4s - loss: 0.0840 - acc: 0.9737
Epoch 22/28
4s - loss: 0.0822 - acc: 0.9735
Epoch 23/28
4s - loss: 0.0805 - acc: 0.97

In [22]:
# Check the training time for the model
t=time.time()
model.fit(X_train, y_train, nb_epoch=7, batch_size=32, shuffle=True, verbose=2)
t2 = time.time()

print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(model.evaluate(X_test, y_test, batch_size=16, verbose=1)[1], 4))


Epoch 1/7
3s - loss: 0.0673 - acc: 0.9796
Epoch 2/7
3s - loss: 0.0731 - acc: 0.9779
Epoch 3/7
4s - loss: 0.0674 - acc: 0.9782
Epoch 4/7
3s - loss: 0.0757 - acc: 0.9791
Epoch 5/7
3s - loss: 0.0707 - acc: 0.9786
Epoch 6/7
3s - loss: 0.0680 - acc: 0.9801
Epoch 7/7
3s - loss: 0.0650 - acc: 0.9799
27.6 Seconds to train SVC...
  16/1913 [..............................] - ETA: 0s  48/1913 [..............................] - ETA: 0s  64/1913 [>.............................] - ETA: 0s  80/1913 [>.............................] - ETA: 0s  96/1913 [>.............................] - ETA: 0s 128/1913 [=>............................] - ETA: 0s 160/1913 [=>............................] - ETA: 0s

In [23]:
# Finish training with test data
t=time.time()
model.fit(X_test, y_test, nb_epoch=21, batch_size=32, shuffle=True, verbose=2)
t2 = time.time()

print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(model.evaluate(X_test, y_test, batch_size=16, verbose=1)[1], 4))


Epoch 1/21
0s - loss: 0.3137 - acc: 0.9373
Epoch 2/21
0s - loss: 0.2209 - acc: 0.9498
Epoch 3/21
0s - loss: 0.1715 - acc: 0.9571
Epoch 4/21
0s - loss: 0.1135 - acc: 0.9650
Epoch 5/21
0s - loss: 0.1317 - acc: 0.9692
Epoch 6/21
0s - loss: 0.0959 - acc: 0.9712
Epoch 7/21
0s - loss: 0.1094 - acc: 0.9639
Epoch 8/21
0s - loss: 0.1143 - acc: 0.9697
Epoch 9/21
0s - loss: 0.0718 - acc: 0.9754
Epoch 10/21
0s - loss: 0.0773 - acc: 0.9770
Epoch 11/21
0s - loss: 0.0660 - acc: 0.9780
Epoch 12/21
0s - loss: 0.0584 - acc: 0.9833
Epoch 13/21
0s - loss: 0.0743 - acc: 0.9749
Epoch 14/21
0s - loss: 0.0455 - acc: 0.9843
Epoch 15/21
0s - loss: 0.0551 - acc: 0.9827
Epoch 16/21
0s - loss: 0.0522 - acc: 0.9833
Epoch 17/21
0s - loss: 0.0356 - acc: 0.9869
Epoch 18/21
0s - loss: 0.0594 - acc: 0.9848
Epoch 19/21
0s - loss: 0.0544 - acc: 0.9843
Epoch 20/21
0s - loss: 0.0299 - acc: 0.9885
Epoch 21/21
0s - loss: 0.0417 - acc: 0.9880
9.02 Seconds to train SVC...
  16/1913 [..............................] - ETA: 0s

In [24]:
# Save the model

import os
import json
import errno

def deleteFile(file):
    try:
        os.remove(file)
    except OSError as error:
        if error.errno != errno.ENOENT:
            raise


# Save the model and weights
deleteFile('model.json')
deleteFile('model.h5')
jsonString = model.to_json()
with open('model.json', 'w') as outfile:
    json.dump(jsonString, outfile)
model.save_weights('model.h5')


In [25]:
del X_train, X_test, y_train, y_test

In [50]:

del model

In [3]:
model= None
import json
from keras.models import model_from_json
with open('model.json', 'r') as jfile:
    model = model_from_json(json.loads(jfile.read()))
model.compile("adam", "binary_crossentropy")
#weights_file = args.model.replace('json', 'h5')
model.load_weights('model.h5')
model.summary()
from sklearn.externals import joblib
X_scaler= joblib.load('scalar.dat')


Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dropout_3 (Dropout)              (None, 1296)          0           dropout_input_1[0][0]            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 512)           664064      dropout_3[0][0]                  
____________________________________________________________________________________________________
dropout_4 (Dropout)              (None, 512)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 1)             513         dropout_4[0][0]                  
Total params: 664,577
Trainable params: 664,577
Non-trainable params: 0
___________________

In [6]:
def getFeaturesFromColorPatch(patch, orient=12, pix_per_cell=4, cell_per_block=2):
    hog_features = []
    feature_patch= cv2.cvtColor(patch, cv2.COLOR_RGB2YCrCb)
    for channel in range(feature_patch.shape[2]):
        hog_features.append(get_hog_features(feature_patch[:,:,channel], 
                            orient, pix_per_cell, cell_per_block, 
                            vis=False, feature_vec=True))
        #hog_features.append(np.zeros(432))   # <--- short out
    return np.ravel(hog_features)

def scalePatch(patch):
    global X_scaler # use previously defined scaler
    X= patch.astype(np.float64).reshape((1, -1))                       
    # Apply the scaler to X
    return np.array(X_scaler.transform(X))


In [9]:
import numpy as np
import cv2
from moviepy.editor import VideoFileClip
import matplotlib.image as mpimg

def resizePatch(img):
    if len(img[0]) != 16:
        if len(img[0])>16:
            patch= cv2.resize(img, (16,16), interpolation=3) #CV_INTER_AREA
        else:
            patch= cv2.resize(img, (16,16), interpolation=2) #CV_INTER_CUBIC
    else:
        patch= img
    return patch

def isVehicle(img):
    patch= resizePatch(img)
    fpatch= getFeaturesFromColorPatch(patch)
    spatch= scalePatch(fpatch)
    conf= model.predict(spatch)[0,0]
    #conf= svc.predict(spatch)
    #conf=0
    return conf>.5 # return True for vehicle   .2

diagScreen= None
imgGlobal= None

def processStrip(x,y,s):
    global frameCount, imgGlobal
    for i in range(4):
        if isVehicle(imgGlobal[y:y+s, x:x+s, :]):
            diagScreen[y:y+s-1, x:x+s-1, 1]= 255  #200
        else:
            diagScreen[y:y+s-1, x:x+s-1, 0]= 63   #157
            #mpimg.imsave('output_images/cells/'+str(frameCount)+'-'+str(x)+'-'+str(y)+'.PNG',resizePatch(img[y:y+s, x:x+s, :]))
        y= y+s

frameCount= 0
def process(img):
    global diagScreen, frameCount, imgGlobal
    diagScreen= np.zeros_like(img).astype(np.uint8)  #*127
    imgGlobal= img
    #for i in range(15):
    #if frameCount>=42 and frameCount<=60:                  # <-------------------------------------- FRAME
    processStrip( 766,411,14)
    processStrip( 780,410,16)
    processStrip( 796,410,18)
    processStrip( 814,409,20)
    processStrip( 834,409,23)
    processStrip( 857,408,25)
    processStrip( 882,407,28)
    processStrip( 910,406,31)
    processStrip( 941,406,35)
    processStrip( 976,405,38)
    processStrip(1014,403,43)
    processStrip(1057,402,46)
    processStrip(1103,401,52)
    processStrip(1155,398,59)
    processStrip(1214,397,66)

    cv2.putText(img, str(frameCount), (1200,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255))
    result= cv2.addWeighted(img, 1, diagScreen, 0.5, 0) # annotate the original
    #result= diagScreen
    frameCount+= 1
    return result

def procVideo(fileName):
    clip= VideoFileClip(fileName)
    imgName= fileName.split('/')[1]
    project_video_output= 'output_images/'+imgName
    print('Processing video...')
    project_video_clip= clip.fl_image(process)
    project_video_clip.write_videofile(project_video_output, audio=False)

procVideo('video/Black.mp4')

Processing video...
[MoviePy] >>>> Building video output_images/Black.mp4
[MoviePy] Writing video output_images/Black.mp4


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [04:52<00:00,  1.20s/it]  0%|▍                                                                                                                           | 1/251 [00:01<04:52,  1.17s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/Black.mp4 



In [1]:
patchBatch= []
predictionBatch= []
def addPatch(p):
    global patchBatch
    patchBatch.append(p)
    
def processBatch(): # send the batch to the GPU for prediction
    global patchBatch, predictionBatch
    for i in range(len(patchBatch)):
        predictionBatch[i]= isVehicle(patchBatch(i))

def getPrediction(i):
    global predictionBatch
    return predictionBatch[i]

def resetBatch():
    global patchBatch, predictionBatch
    patchBatch= []
    predictionBatch= []
